In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ENTREPOT_PATH = '~/Bureau/Datagrosyst/data_entrepot_&_outils/'

# synth = pd.read_csv(ENTREPOT_PATH + 'synthetise.csv')
conx = pd.read_csv(ENTREPOT_PATH + 'connection_synthetise.csv')
noeud = pd.read_csv(ENTREPOT_PATH + 'noeuds_synthetise.csv')

In [ ]:
conx = conx[['id', 'frequence_source','culture_absente','source_noeuds_synthetise_id','cible_noeuds_synthetise_id']]\
    .rename(columns={'id' : 'conx_id',
                     'frequence_source' : 'freq',
                     'culture_absente' : 'abs',
                     'source_noeuds_synthetise_id' : 'nd_prec',
                     'cible_noeuds_synthetise_id' : 'nd_suiv'})
noeud = noeud[['id', 'rang', 'fin_cycle','memecampagne_noeudprecedent', 'synthetise_id']]\
    .rename(columns={'id' : 'nd_id',
                     'fin_cycle' : 'end',
                     'rang' : 'rang',
                     'memecampagne_noeudprecedent' : 'sameyear',
                     'synthetise_id' : 'synth_id'})

In [ ]:
df = conx.merge(noeud[['nd_id','rang','end']].add_suffix('_prec'), left_on='nd_prec', right_on='nd_id_prec')\
    .drop('nd_id_prec', axis=1)
df = df.merge(noeud.add_suffix('_suiv'), left_on='nd_suiv', right_on='nd_id_suiv').\
    rename(columns={'synth_id_suiv' : 'synth_id'})\
        .drop('nd_id_suiv', axis=1)

In [ ]:
def number_node_rank0(dfgrp):
    return len(dfgrp.loc[dfgrp['rang']==0,'rang'])

def number_node_end(dfgrp):
    return len(dfgrp.loc[dfgrp['end']=='t','end'])

def noeud_end_on_rank0(dfgrp):
    return any(dfgrp.loc[dfgrp['end']=='t','rang']==0)

noeud_test = noeud.groupby('synth_id').apply(
    lambda dfgrp: pd.Series({
        'nb_noeud_de_rang1': number_node_rank0(dfgrp),
        'nb_noeud_finaux' : number_node_end(dfgrp),
        'noeud_finaux_en_rang1' : noeud_end_on_rank0(dfgrp)
        }), include_groups=False)

In [ ]:
print(len(noeud_test.loc[noeud_test['nb_noeud_de_rang1']==0,'nb_noeud_de_rang1']))
print(len(noeud_test.loc[noeud_test['nb_noeud_de_rang1']>1,'nb_noeud_de_rang1']))
print(len(noeud_test.loc[noeud_test['nb_noeud_finaux']==0,'nb_noeud_finaux']))
print(len(noeud_test.loc[noeud_test['noeud_finaux_en_rang1']>0,'noeud_finaux_en_rang1']))

list_bad_synth = noeud_test.loc[(noeud_test['nb_noeud_de_rang1']==0) | (noeud_test['nb_noeud_de_rang1']>1) | \
                                (noeud_test['nb_noeud_finaux']==0) | (noeud_test['noeud_finaux_en_rang1']>0)].index

In [ ]:
# noeud suivant est forcement sur le rang suivant (pas de 'trou') OU au moins sur un rang supérieur --- attention si des rang sont complétement vide on les enevle
# somme de sortie du noeuf doit faire 100%
# pas de connexion qui sont à 0% (apparament mais je sias pas pk)
